In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from sklearn.utils import shuffle
from torch.autograd import Variable

In [2]:
df=pd.read_csv('ratings.csv',names=['userID','movieID','rating','time'])
df.drop('time',axis=1,inplace=True)
users=[k for k,v in df['userID'].value_counts().iteritems() if v>2]
movies=[k for k,v in df['movieID'].value_counts().iteritems() if v>10]
df=df[(df['userID'].isin(users)) & (df['movieID'].isin(movies))]
#df=df.sample(frac=1).reset_index(drop=True)
df=df.pivot(index='userID',columns='movieID',values='rating')
df.reset_index(drop=True,inplace=True)
print ('Dataframe size: {}'.format(df.shape))

Dataframe size: (240447, 14277)


In [3]:
train_df=(df.loc[:220000-1])
print ('Train dataframe size: {}'.format(train_df.shape))
test_df=df.loc[230000:].reset_index(drop=True)
print ('Test dataframe size: {}'.format(test_df.shape))
val_df=df.loc[220000:230000-1].reset_index(drop=True)
print ('Validation dataframe size: {}'.format(val_df.shape))

Train dataframe size: (220000, 14277)
Test dataframe size: (10447, 14277)
Validation dataframe size: (10000, 14277)


In [4]:
class Autorec(nn.Module):
    def __init__(self, hidden_size_1, hidden_size_2, dropout, input_size):
        super(Autorec, self).__init__()
        self.input_size=input_size
        self.hidden_size_1=hidden_size_1
        self.hidden_size_2=hidden_size_2
        
        self.encoder_l1=nn.Linear(self.input_size, self.hidden_size_1)
        self.encoder_l2=nn.Linear(self.hidden_size_1, self.hidden_size_2)
        self.decoder_l1=nn.Linear(self.hidden_size_2, self.hidden_size_1)
        self.decoder_l2=nn.Linear(self.hidden_size_1, self.input_size)
        self.drop = nn.Dropout(dropout)
        #self.sigmoid=nn.LogSigmoid()
        
        
    def forward(self, input_ratings):
        #input_ratings=F.normalize(input_ratings)
        enc_out = F.relu(self.encoder_l2(F.relu(self.encoder_l1(input_ratings))))
        enc_out=self.drop(enc_out)
        dec_out = self.decoder_l2(F.relu(self.decoder_l1(enc_out)))
        return dec_out

In [5]:
def train_minibatch(input_ratings, autorec, optimizer,criterion,r):
    optimizer.zero_grad()
    input_ratings=input_ratings.type(torch.cuda.FloatTensor)
    mask=input_ratings!=0
    mask=mask.type(torch.cuda.FloatTensor)
    output_ratings=autorec(input_ratings)
    #loss=torch.mean(torch.sum((output_ratings-input_ratings)**2,-1)/torch.sum(mask,-1))
    loss1=criterion(input_ratings, output_ratings*mask)
    loss1.backward()
    optimizer.step()
    loss2=0
    if r==True:
        optimizer.zero_grad()
        input_ratings=Variable(output_ratings).detach()
        mask=input_ratings!=0
        mask=mask.type(torch.cuda.FloatTensor)
        output_ratings=autorec(input_ratings)
    #loss=torch.mean(torch.sum((output_ratings-input_ratings)**2,-1)/torch.sum(mask,-1))
        loss2=criterion(input_ratings, output_ratings*mask)
        loss2.backward()
        optimizer.step()
    return torch.sqrt(loss1+loss2)

In [6]:
def validation(input_ratings, autorec):
    with torch.no_grad():
        input_ratings=input_ratings.type(torch.cuda.FloatTensor)
        mask=input_ratings!=0
        mask=mask.type(torch.cuda.FloatTensor)
        output_ratings=autorec(input_ratings)*mask
        loss=0
        for i in range(output_ratings.size(0)):
            indices=torch.nonzero(mask[i])
            l=0
            for idx in indices:
                l+=(input_ratings[i][idx]-output_ratings[i][idx])**2
            loss+=l/indices.size(0)
        #loss=torch.mean(torch.sum((output_ratings-input_ratings)**2,-1)/torch.sum(mask,-1))
        return (torch.sqrt(loss/mask.size(0))).item()

In [10]:
autorec=Autorec(hidden_size_1=1024, hidden_size_2=512, dropout=0.5, input_size=train_df.shape[1])
optimizer=optim.Adam(autorec.parameters())
device=torch.device('cuda')
criterion=nn.MSELoss()
autorec=autorec.to(device)

In [8]:
#val_mask=val_df.copy()
#val_mask[~val_mask.isnull()] = 1  # not nan
#val_mask[val_mask.isnull()] = 0   # nan
val_df[val_df.isnull()] = 0   # nan

In [9]:
input_users_val=torch.from_numpy(val_df.values).to(device).detach()
#mask_val=torch.from_numpy(val_mask.values).to(device).detach()

In [15]:
checkpoint = torch.load('model_autorec_DRF.pth')
autorec.load_state_dict(checkpoint['autorec_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
vl=checkpoint['val_loss']
tl=checkpoint['training_loss']
autorec.eval()

Autorec(
  (encoder_l1): Linear(in_features=14277, out_features=1024, bias=True)
  (encoder_l2): Linear(in_features=1024, out_features=512, bias=True)
  (decoder_l1): Linear(in_features=512, out_features=1024, bias=True)
  (decoder_l2): Linear(in_features=1024, out_features=14277, bias=True)
  (drop): Dropout(p=0.5)
)

In [14]:
#num_batches=60
#val_benchmark=10
#vl=[]
#tl=[]
#counter=0

for batch in range(38,90):
    running_loss=0
    counter=0
    #train_df = shuffle(train_df)
    while counter <train_df.shape[0]:
        tdf=(train_df.loc[counter:counter+99].copy())
        #train_mask=tdf.copy()
        #train_mask[~train_mask.isnull()] = 1  # not nan
        #train_mask[train_mask.isnull()] = 0   # nan
        tdf[tdf.isnull()] = 0
        
        assert tdf.shape[0]==100
        input_users=Variable(torch.from_numpy(tdf.values)).to(device)
        #input_mask=torch.from_numpy(train_mask.values).to(device)
        if(batch<40):
            r=False
        elif batch==40:
            r=True
            val_benchmark=1
        loss=train_minibatch(input_users, autorec, optimizer,criterion,r)
        running_loss+=loss.item()
        tl.append(loss.item())
        
        if (counter+100)%44000==0:
            val_rmse=validation(input_users_val, autorec)
            vl.append(val_rmse)
            print ('Batch: {} | Step: {}/{} | Training Loss: {} | Validation RMSE: {}'.format(batch+1,int((counter+100)/44000),5,running_loss,round(val_rmse,4) ))
            running_loss=0
            if(val_rmse<val_benchmark):
                print ('%---Saving the model---%')
                torch.save({
                    'step':counter+1,
                    'autorec_state_dict': autorec.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'batch':batch,
                    'val_loss':vl,
                    'training_loss':tl
                   },'model_autorec_DRF.pth')
                val_benchmark=val_rmse
        counter=counter+100

Batch: 39 | Step: 1/5 | Training Loss: 6260.594717979431 | Validation RMSE: 0.9356
%---Saving the model---%
Batch: 39 | Step: 2/5 | Training Loss: 6258.9457149505615 | Validation RMSE: 0.9369
Batch: 39 | Step: 3/5 | Training Loss: 6045.901337623596 | Validation RMSE: 0.937
Batch: 39 | Step: 4/5 | Training Loss: 6400.710886001587 | Validation RMSE: 0.9368
Batch: 39 | Step: 5/5 | Training Loss: 6376.781566619873 | Validation RMSE: 0.9315
%---Saving the model---%
Batch: 40 | Step: 1/5 | Training Loss: 5800.9616355896 | Validation RMSE: 0.9337
Batch: 40 | Step: 2/5 | Training Loss: 5733.624027252197 | Validation RMSE: 0.9352
Batch: 40 | Step: 3/5 | Training Loss: 5571.05485534668 | Validation RMSE: 0.9389
Batch: 40 | Step: 4/5 | Training Loss: 5655.099016666412 | Validation RMSE: 0.9345
Batch: 40 | Step: 5/5 | Training Loss: 6271.239443778992 | Validation RMSE: 0.9366
Batch: 41 | Step: 1/5 | Training Loss: 490700.85137939453 | Validation RMSE: 0.9982
%---Saving the model---%
Batch: 41 | St

Batch: 58 | Step: 1/5 | Training Loss: 351128.60974121094 | Validation RMSE: 0.9682
Batch: 58 | Step: 2/5 | Training Loss: 349647.9898071289 | Validation RMSE: 0.9583
Batch: 58 | Step: 3/5 | Training Loss: 345712.37268066406 | Validation RMSE: 0.9629
Batch: 58 | Step: 4/5 | Training Loss: 347426.19299316406 | Validation RMSE: 0.9639
Batch: 58 | Step: 5/5 | Training Loss: 346629.40423583984 | Validation RMSE: 0.9636
Batch: 59 | Step: 1/5 | Training Loss: 349926.7329711914 | Validation RMSE: 0.9606
Batch: 59 | Step: 2/5 | Training Loss: 349250.59686279297 | Validation RMSE: 0.9676
Batch: 59 | Step: 3/5 | Training Loss: 347017.5023803711 | Validation RMSE: 0.9718
Batch: 59 | Step: 4/5 | Training Loss: 346055.09588623047 | Validation RMSE: 0.9604
Batch: 59 | Step: 5/5 | Training Loss: 347311.3933105469 | Validation RMSE: 0.9694
Batch: 60 | Step: 1/5 | Training Loss: 351252.9288330078 | Validation RMSE: 0.9656
Batch: 60 | Step: 2/5 | Training Loss: 347121.66015625 | Validation RMSE: 0.9776


Batch: 77 | Step: 5/5 | Training Loss: 340232.3118286133 | Validation RMSE: 0.9893
Batch: 78 | Step: 1/5 | Training Loss: 343598.92108154297 | Validation RMSE: 0.9857
Batch: 78 | Step: 2/5 | Training Loss: 342096.1089477539 | Validation RMSE: 0.982
Batch: 78 | Step: 3/5 | Training Loss: 340375.6696166992 | Validation RMSE: 0.9727
Batch: 78 | Step: 4/5 | Training Loss: 339191.44415283203 | Validation RMSE: 0.9871
Batch: 78 | Step: 5/5 | Training Loss: 341725.2710571289 | Validation RMSE: 1.0079
Batch: 79 | Step: 1/5 | Training Loss: 342537.02166748047 | Validation RMSE: 0.9994
Batch: 79 | Step: 2/5 | Training Loss: 342587.9801635742 | Validation RMSE: 0.9885
Batch: 79 | Step: 3/5 | Training Loss: 340039.18811035156 | Validation RMSE: 0.9752
Batch: 79 | Step: 4/5 | Training Loss: 339368.3359375 | Validation RMSE: 0.9759
Batch: 79 | Step: 5/5 | Training Loss: 339881.6262817383 | Validation RMSE: 1.0013
Batch: 80 | Step: 1/5 | Training Loss: 343071.06689453125 | Validation RMSE: 1.0082
Bat

KeyboardInterrupt: 

In [16]:
#test_mask=test_df.copy()
#test_mask[~test_mask.isnull()] = 1  # not nan
#test_mask[test_mask.isnull()] = 0   # nan
test_df[test_df.isnull()] = 0   # nan

In [17]:
input_users_test=torch.from_numpy(test_df.values).to(device).detach()
#mask_test=torch.from_numpy(test_mask.values).to(device).detach()
#print ('RMSE: {}, Precision@10: {}'.format(validation(input_users_test, mask_test, autorec)))

In [18]:
rmse=validation(input_users_test, autorec)
print ('RMSE: {}'.format(rmse))

RMSE: 0.9381546974182129
